In [ ]:
import base64
import os

import pandas as pd
import numpy as np
import pandas as pd
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import applications, optimizers
from pandas import DataFrame
from IPython.display import HTML

In [2]:
img_size = 299
batch_size = 20
epochs = 100
train_steps_per_epoch = 2

In [ ]:
train_datagen=ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        '../input/prepared_train/prepared_train/',
        classes=['cleaned', 'dirty'],
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary')

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(  
        '../input/platesv2/plates/plates',
        classes=['test'],
        target_size = (img_size, img_size),
        batch_size = 1,
        shuffle = False)    

In [ ]:
base_model = applications.InceptionResNetV2(weights='imagenet', 
                          include_top=False, 
                          input_shape=(img_size, img_size, 3))
base_model.trainable = False
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer = optimizers.rmsprop(lr=1e-4, decay=1e-5), metrics=['accuracy'])

In [ ]:
%%time
model.fit_generator(
        train_generator,
        steps_per_epoch=train_steps_per_epoch,
        epochs=epochs)

In [6]:
test_generator.reset()
predict = model.predict_generator(test_generator, steps = len(test_generator.filenames))

In [7]:
d = {'id': [n for n in range(len(predict))], 
     'label': ['dirty' if x > 0.9995 else 'cleaned' for x in predict]}
result_df = DataFrame(data=d)
result_df.to_csv('data.csv', index=False)